In [1]:
%load_ext autoreload
%autoreload 2
import mycode.vap as vap

In [2]:
# The goal is to check if document id changes format with high trace level.

In [2]:
from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.639.59",
)
# Start a docker container and deploy the application package
client = vespa_docker.deploy(
    application_package=vap.demo_application_package(),
)

Application is up!
Finished deployment.


In [31]:
from vespa.package import DocumentSummary, Summary

app = vap.demo_application_package()
app.get_schema().add_document_summary(
    DocumentSummary(
        name="demo",
        summary_fields=[Summary(name="id", fields=[('source', "id")])]
    )
)

In [23]:
vap.redeploy(vespa_docker, app)

Vespa(http://localhost, 8080)

In [33]:
# Create and feed 1 dummy doc
vap.feed(client, docs=[{'id': 1}])
query = {
    'yql': 'select * from sources * where true limit 1',
    'ranking.profile': 'fields',
    'presentation.summary': 'demo'
}

In [34]:
client.query(body=query).json['root']['children'][0]['id']

'index:test_content/0/c4ca42388ce70a10b392b401'

In [35]:
client.query(body={
    **query,
    'trace.level': 5,
}).json['root']['children'][0]['id']

'index:test_content.num0/0/c4ca42388ce70a10b392b401'

In [38]:
# side by side
print(Out[34])
print(Out[35])

index:test_content/0/c4ca42388ce70a10b392b401
index:test_content.num0/0/c4ca42388ce70a10b392b401


In [37]:
Out[34] == Out[35]

False